In [60]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import pairwise_distances
import plotly.express as px
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score as ss


import joblib
import warnings
warnings.filterwarnings(action='ignore') 

# Train
## Data Load -> Scailing -> SVD -> DBSCAN -> get Best EPS, Min_samples

In [61]:
train_df = pd.read_csv("./dataset/train_data.csv")


In [62]:
features = train_df.columns[:-1]
features

Index(['air_inflow', 'air_end_temp', 'out_pressure', 'motor_current',
       'motor_rpm', 'motor_temp', 'motor_vibe'],
      dtype='object')

In [63]:
X_train = train_df[features]
y_train = train_df[['type']]


In [64]:
sc = StandardScaler()
X_train[features]= sc.fit_transform(X_train)
X_train.head(2)

,air_inflow,air_end_temp,out_pressure,motor_current,motor_rpm,motor_temp,motor_vibe
0,-0.448257,-0.917212,-1.110223e-16,-0.511748,-1.007046,-0.910351,-0.331821
1,0.699749,1.178320,-1.110223e-16,0.802045,1.118785,1.172883,0.188462


In [65]:
train_df = X_train
train_df['type'] = y_train
train_df.head(2)

,air_inflow,air_end_temp,out_pressure,motor_current,motor_rpm,motor_temp,motor_vibe,type
0,-0.448257,-0.917212,-1.110223e-16,-0.511748,-1.007046,-0.910351,-0.331821,0
1,0.699749,1.178320,-1.110223e-16,0.802045,1.118785,1.172883,0.188462,0


In [66]:
g1 = ['air_inflow','motor_current']
g2 = ['air_end_temp', 'motor_rpm', 'motor_temp']
g3 = ['motor_vibe', 'out_pressure']

In [67]:
g1_train = train_df[g1]
g2_train = train_df[g2]
g3_train = train_df[g3]
y_train = train_df[['type']]
combined_train = [g1_train, g2_train, g3_train]

In [68]:
svd= TruncatedSVD(n_components=1, n_iter=7, random_state=42)
svd_list = []
for i in combined_train:
    result = svd.fit_transform(i)
    svd_list.append(result)
    total_var = svd.explained_variance_ratio_.sum() * 100
    print(f"{i}total_var : {total_var}")
    
flat_svd_list = []
for i in svd_list:
    result = [j for sub in i for j in sub]
    flat_svd_list.append(result)
svd_train_df = pd.DataFrame({"PC1": flat_svd_list[0], "PC2": flat_svd_list[1], "PC3": flat_svd_list[2]})
svd_train_df = pd.concat([svd_train_df, y_train], axis=1)
svd_train_df.info()

      air_inflow  motor_current
0      -0.448257      -0.511748
1       0.699749       0.802045
2      -0.182053      -0.202966
3       0.200616       0.230799
4      -0.190372      -0.208848
...          ...            ...
2458    0.125746       0.149927
2459   -0.073907      -0.084600
2460   -0.781013      -0.889638
2461   -0.764375      -0.871259
2462    0.491777       0.566048

[2463 rows x 2 columns]total_var : 99.72924966859078
      air_end_temp  motor_rpm  motor_temp
0        -0.917212  -1.007046   -0.910351
1         1.178320   1.118785    1.172883
2        -0.425427  -0.508305   -0.421612
3         0.266970   0.194005    0.265958
4        -0.434598  -0.517029   -0.430592
...            ...        ...         ...
2458      0.137432   0.063140    0.137680
2459     -0.236279  -0.316369   -0.234326
2460     -1.520194  -1.619204   -1.510693
2461     -1.490389  -1.588669   -1.481189
2462      0.801170   0.736368    0.798312

[2463 rows x 3 columns]total_var : 99.4720531485624
     

In [69]:
tdf0_train = svd_train_df[svd_train_df.type == 0]
tdf1_train = svd_train_df[svd_train_df.type == 1]
tdf2_train = svd_train_df[svd_train_df.type == 2]
tdf3_train = svd_train_df[svd_train_df.type == 3]
tdf4_train = svd_train_df[svd_train_df.type == 4]
tdf5_train = svd_train_df[svd_train_df.type == 5]
tdf6_train = svd_train_df[svd_train_df.type == 6]
tdf7_train = svd_train_df[svd_train_df.type == 7]
tdf_train = [tdf0_train, tdf1_train, tdf2_train, tdf3_train, tdf4_train, tdf5_train, tdf6_train, tdf7_train]

In [70]:
def draw_chart(df):
    fig = px.scatter_3d(
        df, x='PC1', y='PC2', z='PC3', color='type',symbol='type', opacity=0.5, size_max=10,height=600
    )
    fig.show()

In [71]:
# for tdf in tdf_train:
#     print(tdf.type.unique())
#     draw_chart(tdf)

In [72]:
epsilons = np.linspace(0.1, 2, num=50)
epsilons

array([0.1       , 0.13877551, 0.17755102, 0.21632653, 0.25510204,
       0.29387755, 0.33265306, 0.37142857, 0.41020408, 0.44897959,
       0.4877551 , 0.52653061, 0.56530612, 0.60408163, 0.64285714,
       0.68163265, 0.72040816, 0.75918367, 0.79795918, 0.83673469,
       0.8755102 , 0.91428571, 0.95306122, 0.99183673, 1.03061224,
       1.06938776, 1.10816327, 1.14693878, 1.18571429, 1.2244898 ,
       1.26326531, 1.30204082, 1.34081633, 1.37959184, 1.41836735,
       1.45714286, 1.49591837, 1.53469388, 1.57346939, 1.6122449 ,
       1.65102041, 1.68979592, 1.72857143, 1.76734694, 1.80612245,
       1.84489796, 1.88367347, 1.92244898, 1.96122449, 2.        ])

In [73]:
min_samples = np.arange(2, 20, step=1)
min_samples

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19])

In [74]:
epsilons = epsilons
min_samples = min_samples
import itertools
combinations = list(itertools.product(epsilons, min_samples))
N = len(combinations)

In [75]:
def get_scores_and_labels(combinations, X):
  scores = []
  all_labels_list = []

  for i, (eps, num_samples) in enumerate(combinations):
    dbscan_cluster_model = DBSCAN(eps=eps, min_samples=num_samples).fit(X)
    labels = dbscan_cluster_model.labels_
    labels_set = set(labels)
    num_clusters = len(labels_set)
    if -1 in labels_set:
      num_clusters -= 1
    
    if (num_clusters < 2) or (num_clusters > 50):
      scores.append(-10)
      all_labels_list.append('bad')
      c = (eps, num_samples)
      print(f"Combination {c} on iteration {i+1} of {N} has {num_clusters} clusters. Moving on")
      continue
    try:
        scores.append(ss(X, labels))
        all_labels_list.append(labels)
        print(f"Index: {i}, Score: {scores[-1]}, Labels: {all_labels_list[-1]}, NumClusters: {num_clusters}")
    except:
        scores.append('ss Failure')

  best_index = np.argmax(scores)
  best_parameters = combinations[best_index]
  best_labels = all_labels_list[best_index]
  best_score = scores[best_index]

  return {'best_epsilon': best_parameters[0],
          'best_min_samples': best_parameters[1], 
          'best_labels': best_labels,
          'best_score': best_score}

In [76]:
results = []

for tdf in tdf_train:
    X_train = tdf.iloc[:,:-1]
    y_train = tdf[['type']]
    
    result = get_scores_and_labels(combinations, tdf)
    results.append(result)
    
    tdf['cluster'] = result['best_labels']
    
    print(tdf.type.unique())
#     fig = px.scatter_3d(
#     tdf, x='PC1', y='PC2', z='PC3', color='cluster',symbol='cluster', opacity=0.5, size_max=10,height=600,
#     )
#     fig.show()
    print("="*70)
    

Combination (0.1, 2) on iteration 1 of 900 has 1 clusters. Moving on
Combination (0.1, 3) on iteration 2 of 900 has 1 clusters. Moving on
Combination (0.1, 4) on iteration 3 of 900 has 1 clusters. Moving on
Combination (0.1, 5) on iteration 4 of 900 has 1 clusters. Moving on
Combination (0.1, 6) on iteration 5 of 900 has 1 clusters. Moving on
Combination (0.1, 7) on iteration 6 of 900 has 1 clusters. Moving on
Index: 6, Score: 0.19505069403637373, Labels: [ 0  1  0  0  0  0  1  1  0  1 -1  0  0  1  1  1  0  0  1  0  0  0  1  1
  0  1  0  1  0  0  0  1  1  0  0  1  0  0  0  0  1  0  0  0  0  1  0  1
  0  0  0  1  0  1  0  1  1  0  1  0  0  1  0  0  1  0  0  0  0  0  0  0
  1  0  1  0  1  0  1  0  1  0  0  1  0  1  1  0  1  0  0  1  0  1  0  0
  0  1  1  1  0  1  1  0  1  0  0  1  1  0  0  1  1  1  1  0  0  1  1  0
  0  0  0  1  0  1  0  0 -1  0  1  1  0  1  0  0  1  0  1  0  0  1  0  1
  0 -1  0  0  0  1  0  1  1  0  0  0  1  0  1  0  1  1  1  1  0  0  0  1
  0  0  1  0  0  0  0  0  0  

Index: 13, Score: 0.3064500014916349, Labels: [ 7 -1 -1 10 -1  0  2  1  0  2 -1 -1 10  3  8  2  4 -1  3  4  0  5  2  3
  6  8  6  3  7  4  5 -1 -1  7  5  1 -1 -1  0 10  8  6  0  9  5  1  7  8
 -1 -1 -1 -1  9  1  4  2  2  7  2  0  4  8  6  6  3  9  0 -1  5 11 -1  0
  3 -1  1 -1 -1  4  2  6  3  0  0  1  5 -1  1 11 -1 -1  9  3  0  8 -1 -1
  0  8 -1  1 -1  3  3 -1 -1  5  9  2  3 -1 -1  1  3  3  1  0  9  2  2  0
 -1 -1  5 -1 11 -1  4  0 -1 11  3  1 -1  3 -1 10  3  9 -1  5  0  8  0  2
  5 -1  5  5 -1 -1  4  1  8  5 -1  5  8  4  8  0  2  8  3  1 11  0  0  8
  0  0  3  0  6 -1 -1 -1  0  1 -1  8  0 -1  0  7 11  3  1  0 10 -1  5  3
  0 -1  0  8  0 10  1  3  0  3  7 11  2 -1  2  0  1  3  0  9  0  9  1 -1
  7 -1 10  4  7  0  6  6  0  6  5  6  4  9  3  0  6  0  8  1  3  0  8 10
  3  3  7  7 10 10  8  1 -1  5  7  4  2  3 -1  1 -1 -1  0 -1 -1 -1  7 -1
 10  1  9  2  5 -1  9  2  9 -1  2  7  3  1  6  6 -1  2  3  2 -1  3 10  2
  3 10  2  0 -1  5  2 -1 -1  1  2  8  7  3  0  3 -1  4  4  2 -1  4  2  0
  6  

Index: 30, Score: 0.4156607605882992, Labels: [0 1 0 0 0 0 1 1 0 1 0 0 0 2 1 1 0 0 2 0 0 0 1 2 0 1 0 2 0 0 0 1 2 0 0 1 0
 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 1 0 0 2 0 0 0 0 0 0 0 2 0
 1 0 1 0 1 0 2 0 0 1 0 1 1 0 1 0 0 2 0 1 0 0 0 1 1 1 0 2 2 0 1 0 0 1 2 0 0
 1 2 2 1 0 0 1 1 0 0 0 0 1 0 1 0 0 0 0 2 1 0 2 0 0 2 0 1 0 0 1 0 1 0 0 0 0
 0 1 0 1 1 0 0 0 1 0 1 0 1 1 2 1 0 0 0 1 0 0 2 0 0 0 0 0 0 1 1 1 0 1 0 0 0
 2 1 0 0 0 0 2 0 1 0 1 0 0 1 2 0 2 0 0 1 0 1 0 1 2 0 0 0 0 1 2 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 2 0 0 0 1 1 2 0 1 0 2 2 0 0 0 0 1 1 1 0 0 0 1 2 0 1 0 1 0
 2 0 0 0 0 0 1 0 1 0 1 0 1 0 1 1 0 2 1 0 0 0 1 2 1 0 2 0 1 2 0 1 0 1 0 1 1
 1 1 1 1 0 2 0 2 1 0 0 1 0 0 1 0 0 0 0 1 1 1 2 1 0 0 0 0 0 0 0 1 1 0 0 0 1
 0 0 1 0 2 0 1 0 0 0 2 2 1 0 1 0 1 0 1 0 1 1 0 0 0 0 0 0 1 0 1 1 2 0 0 2 0
 1 1 0 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 0 2 0 0 1 0 1 1 0 2 0 0 2 1
 2 0 0 2 0 0 2 2 1 2 1 0 0 1 0 0 2 0 1 0 0 0 1 0 0], NumClusters: 3
Index: 31, Score: 0.47754152845856945, Labels: [ 0  2  1  1  

Combination (0.2163265306122449, 2) on iteration 55 of 900 has 1 clusters. Moving on
Combination (0.2163265306122449, 3) on iteration 56 of 900 has 1 clusters. Moving on
Combination (0.2163265306122449, 4) on iteration 57 of 900 has 1 clusters. Moving on
Combination (0.2163265306122449, 5) on iteration 58 of 900 has 1 clusters. Moving on
Combination (0.2163265306122449, 6) on iteration 59 of 900 has 1 clusters. Moving on
Combination (0.2163265306122449, 7) on iteration 60 of 900 has 1 clusters. Moving on
Combination (0.2163265306122449, 8) on iteration 61 of 900 has 1 clusters. Moving on
Combination (0.2163265306122449, 9) on iteration 62 of 900 has 1 clusters. Moving on
Combination (0.2163265306122449, 10) on iteration 63 of 900 has 1 clusters. Moving on
Combination (0.2163265306122449, 11) on iteration 64 of 900 has 1 clusters. Moving on
Combination (0.2163265306122449, 12) on iteration 65 of 900 has 1 clusters. Moving on
Combination (0.2163265306122449, 13) on iteration 66 of 900 ha

Combination (0.4102040816326531, 12) on iteration 155 of 900 has 1 clusters. Moving on
Combination (0.4102040816326531, 13) on iteration 156 of 900 has 1 clusters. Moving on
Combination (0.4102040816326531, 14) on iteration 157 of 900 has 1 clusters. Moving on
Combination (0.4102040816326531, 15) on iteration 158 of 900 has 1 clusters. Moving on
Combination (0.4102040816326531, 16) on iteration 159 of 900 has 1 clusters. Moving on
Combination (0.4102040816326531, 17) on iteration 160 of 900 has 1 clusters. Moving on
Combination (0.4102040816326531, 18) on iteration 161 of 900 has 1 clusters. Moving on
Combination (0.4102040816326531, 19) on iteration 162 of 900 has 1 clusters. Moving on
Combination (0.44897959183673475, 2) on iteration 163 of 900 has 1 clusters. Moving on
Combination (0.44897959183673475, 3) on iteration 164 of 900 has 1 clusters. Moving on
Combination (0.44897959183673475, 4) on iteration 165 of 900 has 1 clusters. Moving on
Combination (0.44897959183673475, 5) on ite

...

Combination (0.7204081632653061, 16) on iteration 303 of 900 has 1 clusters. Moving on
Combination (0.7204081632653061, 17) on iteration 304 of 900 has 1 clusters. Moving on
Combination (0.7204081632653061, 18) on iteration 305 of 900 has 1 clusters. Moving on
Combination (0.7204081632653061, 19) on iteration 306 of 900 has 1 clusters. Moving on
Combination (0.7591836734693878, 2) on iteration 307 of 900 has 1 clusters. Moving on
Combination (0.7591836734693878, 3) on iteration 308 of 900 has 1 clusters. Moving on
Combination (0.7591836734693878, 4) on iteration 309 of 900 has 1 clusters. Moving on
Combination (0.7591836734693878, 5) on iteration 310 of 900 has 1 clusters. Moving on
Combination (0.7591836734693878, 6) on iteration 311 of 900 has 1 clusters. Moving on
Combination (0.7591836734693878, 7) on iteration 312 of 900 has 1 clusters. Moving on
Combination (0.7591836734693878, 8) on iteration 313 of 900 has 1 clusters. Moving on
Combination (0.7591836734693878, 9) on iteration 3

Combination (1.0306122448979593, 10) on iteration 441 of 900 has 1 clusters. Moving on
Combination (1.0306122448979593, 11) on iteration 442 of 900 has 1 clusters. Moving on
Combination (1.0306122448979593, 12) on iteration 443 of 900 has 1 clusters. Moving on
Combination (1.0306122448979593, 13) on iteration 444 of 900 has 1 clusters. Moving on
Combination (1.0306122448979593, 14) on iteration 445 of 900 has 1 clusters. Moving on
Combination (1.0306122448979593, 15) on iteration 446 of 900 has 1 clusters. Moving on
Combination (1.0306122448979593, 16) on iteration 447 of 900 has 1 clusters. Moving on
Combination (1.0306122448979593, 17) on iteration 448 of 900 has 1 clusters. Moving on
Combination (1.0306122448979593, 18) on iteration 449 of 900 has 1 clusters. Moving on
Combination (1.0306122448979593, 19) on iteration 450 of 900 has 1 clusters. Moving on
Combination (1.0693877551020408, 2) on iteration 451 of 900 has 1 clusters. Moving on
Combination (1.0693877551020408, 3) on itera

Combination (1.3020408163265307, 9) on iteration 566 of 900 has 1 clusters. Moving on
Combination (1.3020408163265307, 10) on iteration 567 of 900 has 1 clusters. Moving on
Combination (1.3020408163265307, 11) on iteration 568 of 900 has 1 clusters. Moving on
Combination (1.3020408163265307, 12) on iteration 569 of 900 has 1 clusters. Moving on
Combination (1.3020408163265307, 13) on iteration 570 of 900 has 1 clusters. Moving on
Combination (1.3020408163265307, 14) on iteration 571 of 900 has 1 clusters. Moving on
Combination (1.3020408163265307, 15) on iteration 572 of 900 has 1 clusters. Moving on
Combination (1.3020408163265307, 16) on iteration 573 of 900 has 1 clusters. Moving on
Combination (1.3020408163265307, 17) on iteration 574 of 900 has 1 clusters. Moving on
Combination (1.3020408163265307, 18) on iteration 575 of 900 has 1 clusters. Moving on
Combination (1.3020408163265307, 19) on iteration 576 of 900 has 1 clusters. Moving on
Combination (1.3408163265306123, 2) on itera

Combination (1.5346938775510206, 14) on iteration 679 of 900 has 1 clusters. Moving on
Combination (1.5346938775510206, 15) on iteration 680 of 900 has 1 clusters. Moving on
Combination (1.5346938775510206, 16) on iteration 681 of 900 has 1 clusters. Moving on
Combination (1.5346938775510206, 17) on iteration 682 of 900 has 1 clusters. Moving on
Combination (1.5346938775510206, 18) on iteration 683 of 900 has 1 clusters. Moving on
Combination (1.5346938775510206, 19) on iteration 684 of 900 has 1 clusters. Moving on
Combination (1.573469387755102, 2) on iteration 685 of 900 has 1 clusters. Moving on
Combination (1.573469387755102, 3) on iteration 686 of 900 has 1 clusters. Moving on
Combination (1.573469387755102, 4) on iteration 687 of 900 has 1 clusters. Moving on
Combination (1.573469387755102, 5) on iteration 688 of 900 has 1 clusters. Moving on
Combination (1.573469387755102, 6) on iteration 689 of 900 has 1 clusters. Moving on
Combination (1.573469387755102, 7) on iteration 690 o

Combination (1.806122448979592, 8) on iteration 799 of 900 has 1 clusters. Moving on
Combination (1.806122448979592, 9) on iteration 800 of 900 has 1 clusters. Moving on
Combination (1.806122448979592, 10) on iteration 801 of 900 has 1 clusters. Moving on
Combination (1.806122448979592, 11) on iteration 802 of 900 has 1 clusters. Moving on
Combination (1.806122448979592, 12) on iteration 803 of 900 has 1 clusters. Moving on
Combination (1.806122448979592, 13) on iteration 804 of 900 has 1 clusters. Moving on
Combination (1.806122448979592, 14) on iteration 805 of 900 has 1 clusters. Moving on
Combination (1.806122448979592, 15) on iteration 806 of 900 has 1 clusters. Moving on
Combination (1.806122448979592, 16) on iteration 807 of 900 has 1 clusters. Moving on
Combination (1.806122448979592, 17) on iteration 808 of 900 has 1 clusters. Moving on
Combination (1.806122448979592, 18) on iteration 809 of 900 has 1 clusters. Moving on
Combination (1.806122448979592, 19) on iteration 810 of 

In [77]:
results[0]

{'best_epsilon': 0.13877551020408163,
 'best_min_samples': 13,
 'best_labels': array([0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
        0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 1, 0, 1, 0, 1, 0,

In [78]:
best_eps = [results[i]['best_epsilon'] for i in range(8)]
best_eps

[0.13877551020408163,
 0.8755102040816326,
 0.1,
 0.17755102040816328,
 0.13877551020408163,
 0.17755102040816328,
 0.13877551020408163,
 0.17755102040816328]

In [79]:
best_min_samples = [results[i]['best_min_samples'] for i in range(8)]
best_min_samples

[13, 2, 2, 10, 8, 2, 2, 2]

In [80]:
best_sil_score = [results[i]['best_score'] for i in range(7)]
best_sil_score

[0.5277549447576949,
 0.8260353154884035,
 0.5716122215821983,
 0.5613954395572766,
 0.63725196055693,
 0.6244164542412234,
 0.6120270877318426]

# Test
## Data Load -> Scailing -> SVD -> DBSCAN(with Best EPS, Min_samples) -> Get Label

In [81]:
test_df = pd.read_csv("./dataset/test_data.csv")
X_test = test_df[features]
y_test = test_df[['type']]

In [82]:
sc = StandardScaler()
X_test[features]= sc.fit_transform(X_test)
X_test.head(2)

,air_inflow,air_end_temp,out_pressure,motor_current,motor_rpm,motor_temp,motor_vibe
0,0.349570,0.487848,0.029701,0.387062,0.419055,0.491978,0.208765
1,0.478445,0.713509,0.029701,0.531396,0.647355,0.716757,0.348482


In [83]:
test_df = X_test
test_df['type'] = y_test
test_df.head(2)

,air_inflow,air_end_temp,out_pressure,motor_current,motor_rpm,motor_temp,motor_vibe,type
0,0.349570,0.487848,0.029701,0.387062,0.419055,0.491978,0.208765,0
1,0.478445,0.713509,0.029701,0.531396,0.647355,0.716757,0.348482,0


In [84]:
g1_test = test_df[g1]
g2_test = test_df[g2]
g3_test = test_df[g3]
y_test = test_df[['type']]
combined_test = [g1_test, g2_test, g3_test]

In [85]:
svd = TruncatedSVD(n_components=1)
svd_list = []
for i in combined_test:
    result = svd.fit_transform(i)
    svd_list.append(result)
    total_var =svd.explained_variance_ratio_.sum() * 100
    print(f"{i}total_var : {total_var}")
    
flat_svd_list = []
for i in svd_list:
    result = [j for sub in i for j in sub]
    flat_svd_list.append(result)
svd_test_df = pd.DataFrame({"PC1": flat_svd_list[0], "PC2": flat_svd_list[1], "PC3": flat_svd_list[2]})
svd_test_df = pd.concat([svd_test_df, y_test], axis=1)
svd_test_df.info()

      air_inflow  motor_current
0       0.349570       0.387062
1       0.478445       0.531396
2      -0.329171      -0.392040
3       0.083228       0.081769
4      -0.037055      -0.056520
...          ...            ...
7384    0.014495       0.002423
7385   -0.535371      -0.621765
7386   -0.466638      -0.549220
7387   -0.440863      -0.521260
7388   -0.346355      -0.408664

[7389 rows x 2 columns]total_var : 99.55080635654564
      air_end_temp  motor_rpm  motor_temp
0         0.487848   0.419055    0.491978
1         0.713509   0.647355    0.716757
2        -0.725652  -0.806787   -0.718468
3         0.012350  -0.060812    0.017876
4        -0.202949  -0.278934   -0.197860
...            ...        ...         ...
7384     -0.110842  -0.185868   -0.104848
7385     -1.083715  -1.168869   -1.076305
7386     -0.970885  -1.053991   -0.962624
7387     -0.927134  -1.010367   -0.918702
7388     -0.752132  -0.832962   -0.744304

[7389 rows x 3 columns]total_var : 99.22881044537633
    

In [87]:
tdf0_test = svd_test_df[svd_test_df.type == 0]
tdf1_test = svd_test_df[svd_test_df.type == 1]
tdf2_test = svd_test_df[svd_test_df.type == 2]
tdf3_test = svd_test_df[svd_test_df.type == 3]
tdf4_test = svd_test_df[svd_test_df.type == 4]
tdf5_test = svd_test_df[svd_test_df.type == 5]
tdf6_test = svd_test_df[svd_test_df.type == 6]
tdf7_test = svd_test_df[svd_test_df.type == 7]
tdf_test = [tdf0_test, tdf1_test, tdf2_test, tdf3_test, tdf4_test, tdf5_test, tdf6_test, tdf7_test]

In [88]:
clusters = []
sil_scores = []

for i, tdf in enumerate(tdf_test):
    X_test = tdf.iloc[:,:-1]
    y_test = tdf[['type']]
    
    dbscan = DBSCAN(eps=best_eps[i], min_samples=best_min_samples[i]).fit(X_test)

    try: 
        sil_score = ss(X_test, dbscan.labels_)
        sil_scores.append(sil_score) 
    except:
        sil_scores.append("값없음") 
        
    clusters.extend(dbscan.labels_)

In [89]:
sil_scores

[0.0106926353250772,
 '값없음',
 -0.5703847180674092,
 0.09985686870781159,
 0.8244814455464595,
 -0.7746449898203959,
 -0.6807657835972145,
 -0.04211514473469904]

In [90]:
svd_test_df['label'] = clusters
svd_test_df['label'].value_counts()

 0     7053
-1      215
 3       17
 4       11
 6       11
 7       11
 5       10
 1        9
 2        9
 11       7
 13       7
 8        6
 9        6
 12       3
 14       3
 15       3
 16       3
 17       3
 10       2
Name: label, dtype: int64

In [91]:
# ddf = svd_test_df['label'].value_counts().to_frame()
# ddf

In [92]:
# svd_test_df.info()

In [93]:
# def 결과정리(label):
#     if label in [-1, 18, 13, 12, 10, 17, 16, 11, 15, 14, 8, 9, 6, 2, 1, 5, 4, 7, 3]: 
#         return 1
#     else:
#         return 0

In [94]:
def 결과정리(label):
    if label == 0: 
        return 0
    else:
        return 1

In [95]:
svd_test_df['label'] = svd_test_df['label'].apply(결과정리)
svd_test_df['label'].value_counts()

0    7053
1     336
Name: label, dtype: int64

In [96]:
overall_sil_score = ss(svd_test_df.iloc[:,:-2], svd_test_df['label'])
overall_sil_score

0.09912640077526606

In [97]:
fig = px.scatter_3d(
    svd_test_df, x='PC1', y='PC2', z='PC3', color='label',symbol='label', opacity=0.5, size_max=10,height=800,
)
fig.show()

In [98]:
submit_df = svd_test_df[['type', 'label']]
submit_df['label'].value_counts()

0    7053
1     336
Name: label, dtype: int64

In [99]:
submit_df.to_csv('./submits/submit_20230411_01.csv', index=False)